# Politician Wikipedia File Extractions
This file completes the data acquisition, getting article quality predictions, and combining the datasets sections of Homework 2 using [Wikimedia REST API](https://www.mediawiki.org/wiki/Wikimedia_REST_API) ([API](https://www.mediawiki.org/wiki/API:Info)) and [ORES](https://www.mediawiki.org/wiki/ORES) ([API](https://ores.wikimedia.org)). This method will request the latest revision IDs and predict the article qualities for all politician articles in the politicians_by_country_SEPT.2022.csv file (this file was created by crawling the Wikipedia Category: [Politicians by nationality](https://en.wikipedia.org/wiki/Category:Politicians_by_nationality)). Country and region populations are found in population_by_country_2022.csv (this file This dataset is drawn from the [world population data sheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau).

## Crediting Code

Some code used in this file uses the example notebooks ([1](https://drive.google.com/file/d/1Z8DqXpHmNUJ3RD7e-OOwx2WYJPIYjUPp/view?usp=sharing), [2]())provided by professor McDonald. Both are [licensed CC0](https://drive.google.com/file/d/1rZdBrtCe9XO4IkDWqm0A2RA-HfZCsqHh/view?usp=sharing) and are allowed to be reused.

In [112]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests
import pandas as pd
import numpy as np

First we will bring in the politician data.

In [60]:
# Getting the dinosaur names for the data pull
wiki_politicians = pd.read_csv("politicians_by_country_SEPT.2022.csv")
# This step is needed for the quotation marks to be read correctly
wiki_politicians['name'] = wiki_politicians.name.replace({'Segun “Aeroland” Adewale': 'Segun "Aeroland" Adewale',
                                                          'Nhlanhla “Lux” Dlamini': 'Nhlanhla "Lux" Dlamini'})
wiki_politicians.head()

,name,url,country
0,Shahjahan Noori,https://en.wikipedia.org/wiki/Shahjahan_Noori,Afghanistan
1,Abdul Ghafar Lakanwal,https://en.wikipedia.org/wiki/Abdul_Ghafar_Lak...,Afghanistan
2,Majah Ha Adrif,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,Afghanistan
3,Haroon al-Afghani,https://en.wikipedia.org/wiki/Haroon_al-Afghani,Afghanistan
4,Tayyab Agha,https://en.wikipedia.org/wiki/Tayyab_Agha,Afghanistan


The example relies on some constants that help make the code a bit more readable.

In [70]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<rafrisci@uw.edu>, Ross Friscia1, MSDS DATA 512 - AUTUMN 2022',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = wiki_politicians['name']

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

# The current ORES API endpoint
API_ORES_SCORE_ENDPOINT = "https://ores.wikimedia.org/v3"
# A template for mapping to the URL
API_ORES_SCORE_PARAMS = "/scores/{context}/{revid}/{model}"

# Use some delays so that we do not hammer the API with our requests
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022'
}

# A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
ARTICLE_REVISIONS = { 'Shahjahan Noori':1099689043 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

# This template lists the basic parameters for making an ORES request
ORES_PARAMS_TEMPLATE = {
    "context": "enwiki",        # which WMF project for the specified revid
    "revid" : "",               # the revision to be scored - this will probably change each call
    "model": "articlequality"   # the AI/ML scoring model to apply to the reviewion
}

The API request and ORES request will each be made using one procedure. The idea is to make them reusable. The procedures are parameterized, but both rely on the constants above for the important parameters. The underlying assumption is that these will be used to request data for a set of article pages. Therefore the parameter most likely to change for request_pageinfo_per_article is the article_title and for request_ores_score_per_article is article_revid.

In [71]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    request_template['titles'] = article_title
        
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

def request_ores_score_per_article(article_revid = None, 
                                   endpoint_url = API_ORES_SCORE_ENDPOINT, 
                                   endpoint_params = API_ORES_SCORE_PARAMS, 
                                   request_template = ORES_PARAMS_TEMPLATE,
                                   headers = REQUEST_HEADERS,
                                   features=False):
    # Make sure we have an article revision id
    if not article_revid: return None
    
    # set the revision id into the template
    request_template['revid'] = article_revid
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # the features used by the ML model can sometimes be returned as well as scores
    if features:
        request_url = request_url+"?features=true"
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

Now we will make the API requests for all politicians and output the politicians with their latest revision ID and a list of politicians that there were no revision IDs for.

In [63]:
politician_revisions = {}
no_log = []
for politician in ARTICLE_TITLES:    
    print("Getting page info data for: ",politician)
    info = request_pageinfo_per_article(politician)
    rev_path = list(info['query']['pages'].keys())
    try:
        revision_id = info['query']['pages'][rev_path[0]]['lastrevid']
        politician_revisions[politician] = revision_id
    except KeyError:
        no_log.append(politician)
        print('No log for ', politician)

Getting page info data for:  Shahjahan Noori
Getting page info data for:  Abdul Ghafar Lakanwal
Getting page info data for:  Majah Ha Adrif
Getting page info data for:  Haroon al-Afghani
Getting page info data for:  Tayyab Agha
Getting page info data for:  Ahmadullah Wasiq
Getting page info data for:  Aziza Ahmadyar
Getting page info data for:  Muqadasa Ahmadzai
Getting page info data for:  Mohammad Sarwar Ahmedzai
Getting page info data for:  Amir Muhammad Akhundzada
Getting page info data for:  Nasrullah Baryalai Arsalai
Getting page info data for:  Mohammad Asim Asim
Getting page info data for:  Atiqullah Atifmal
Getting page info data for:  Abdul Rahim Ayoubi
Getting page info data for:  Alhaj Mutalib Baig
Getting page info data for:  Ismael Balkhi
Getting page info data for:  Abdul Baqi Turkistani
Getting page info data for:  Mohammad Ghous Bashiri
Getting page info data for:  Abas Basir
Getting page info data for:  Jan Baz
Getting page info data for:  Ahmad Behzad
Getting page in

Getting page info data for:  Sali Nivica
Getting page info data for:  Thoma Orologa
Getting page info data for:  Bedri Pejani
Getting page info data for:  Pertef Pogoni
Getting page info data for:  Kadri Prishtina
Getting page info data for:  Ymer Prizreni
Getting page info data for:  Avni Rustemi
Getting page info data for:  Chatin Sarachi
Getting page info data for:  Nebi Sefa
Getting page info data for:  Eduard Shalsi
Getting page info data for:  Ylli Vesel Shehu
Getting page info data for:  Athanas Shundi
Getting page info data for:  Xhelal Sveçla
Getting page info data for:  Mehmet Tahsini
Getting page info data for:  Ibrahim Temo
Getting page info data for:  Pavli Terka
Getting page info data for:  Afrim Tole
Getting page info data for:  Sejfi Vllamasi
Getting page info data for:  Syrja Vlora
Getting page info data for:  Visarion Xhuvani
Getting page info data for:  Visar Ymeri
Getting page info data for:  Abdyl Ypi
Getting page info data for:  Xhezair Zaganjori
Getting page info

Getting page info data for:  Amán Rawson
Getting page info data for:  Martín Sabbatella
Getting page info data for:  Juan Pablo Schiavi
Getting page info data for:  Héctor Tizón
Getting page info data for:  Claudio Uberti
Getting page info data for:  Arístides Villanueva
Getting page info data for:  Artashes Abeghyan
Getting page info data for:  Aram Achekbashian
Getting page info data for:  Hovsep Arghutian
Getting page info data for:  Mikael Atabekyan
Getting page info data for:  Tigran Bekzadyan
Getting page info data for:  Vigen Chitechyan
Getting page info data for:  Hayk Chobanyan
Getting page info data for:  Daniel-Bek of Sassun
Getting page info data for:  Gurgen Egiazaryan
Getting page info data for:  Parunak Ferukhan
Getting page info data for:  Hambardzum Galstyan
Getting page info data for:  Gegham Gharibjanian
Getting page info data for:  Gevorg Ghazaryan (politician)
Getting page info data for:  Andrias Ghukasyan
Getting page info data for:  Aram Grigoryan
Getting page in

Getting page info data for:  Anar Karimov
Getting page info data for:  Boris Kevorkov
Getting page info data for:  Gurban Khalilov
Getting page info data for:  Ahmad Kordary
Getting page info data for:  Ilham Madatov
Getting page info data for:  Musa Mammadov
Getting page info data for:  Ramin Mammadov
Getting page info data for:  Elshad Mirbashir oghlu
Getting page info data for:  Fuad Naghiyev
Getting page info data for:  Eldar Namazov
Getting page info data for:  Siyavush Novruzov
Getting page info data for:  Sadig Rahimov
Getting page info data for:  Suleyman Rahimov
Getting page info data for:  Huseyn Rahmanov
Getting page info data for:  Hasan Sayidov
Getting page info data for:  Parviz Shahbazov
Getting page info data for:  Mammad agha Shahtakhtinski
Getting page info data for:  Abid Sharifov
Getting page info data for:  Bakhtiyar Vahabzadeh
Getting page info data for:  Elmir Valiyev
Getting page info data for:  Nurali Yusifbayli
Getting page info data for:  Harold Christie
Gett

Getting page info data for:  Vladimir Vasilievich Rusakevich
Getting page info data for:  Ihar Rynkevich
Getting page info data for:  Aliaksandr Shakutsin
Getting page info data for:  Nikolai Shelyagovich
Getting page info data for:  Jan Stankievič
Getting page info data for:  Mikola Statkevich
Getting page info data for:  Sergei Tikhanovsky
Getting page info data for:  Alaksandar Ćvikievič
Getting page info data for:  Aleksandr Turchin
Getting page info data for:  Franak Viačorka
Getting page info data for:  Yury Voskresensky
Getting page info data for:  Ivan Yermachenka
Getting page info data for:  Manuel Abramowicz
Getting page info data for:  Rudy Aernoudt
Getting page info data for:  Herman Baltia
Getting page info data for:  Jan Béghin
Getting page info data for:  Frank Beke
Getting page info data for:  Gustave Boël
Getting page info data for:  Pol Clovis Boël
Getting page info data for:  Antoine Brun
Getting page info data for:  Fernand Cocq
Getting page info data for:  Émile Co

...

Getting page info data for:  Bertil Zachrisson
Getting page info data for:  Céline Amaudruz
Getting page info data for:  Beat Arnold
Getting page info data for:  Mustafa Atici
Getting page info data for:  Gallus Jacob Baumgartner
Getting page info data for:  Martin Bäumle
Getting page info data for:  Pierre François Bellot
Getting page info data for:  Jacqueline Berenstein-Wavre
Getting page info data for:  Peter Bodenmann
Getting page info data for:  Ulrich Bremi
Getting page info data for:  Achille Casanova
Getting page info data for:  Conradin Cramer
Getting page info data for:  Niklaus Dachselhofer
Getting page info data for:  Sigmund von Erlach
Getting page info data for:  Heinrich Escher
Getting page info data for:  Johann Rudolf Faesch
Getting page info data for:  Peter Falck
Getting page info data for:  Franz Rudolf Frisching
Getting page info data for:  Urs Glutz von Blotzheim
Getting page info data for:  Jürg Grossen
Getting page info data for:  Tim Guldimann
Getting page inf

Getting page info data for:  María do Céu
Getting page info data for:  Brigida Antónia Correia
Getting page info data for:  Isabel da Costa Ferreira
Getting page info data for:  Albina Marçal Freitas
Getting page info data for:  Ángela Freitas
Getting page info data for:  Justino Mota
Getting page info data for:  Maria da Paixão da Costa
Getting page info data for:  Olinda Morais
Getting page info data for:  Manuel Magalhaes de Oliveira
Getting page info data for:  Alfredo Pires
Getting page info data for:  Nino Konis Santana
Getting page info data for:  José Abílio Osório Soares
Getting page info data for:  João da Costa Tavares
Getting page info data for:  Mário Viegas Carrascalão
Getting page info data for:  Maria Terezinha Viegas
Getting page info data for:  Kafui Adjamagbo-Johnson
Getting page info data for:  Angèle Dola Akofa Aguigah
Getting page info data for:  Robert Ajavon
Getting page info data for:  Martin Aku
Getting page info data for:  Claude Améganvi
Getting page info da

Getting page info data for:  Christine Apolot
Getting page info data for:  Florence Akiiki Asiimwe
Getting page info data for:  Charles Ayume
Getting page info data for:  Edith Mary Bataringaya
Getting page info data for:  Flavia Rwabuhoro Kabahenda
Getting page info data for:  John Kabaireho
Getting page info data for:  Joseph Kabuleta
Getting page info data for:  Apollo Kaggwa
Getting page info data for:  Michael Kawalya Kagwa
Getting page info data for:  Robert Mugabe Kakyebezi
Getting page info data for:  Paulo Kavuma
Getting page info data for:  Patrick Kayemba
Getting page info data for:  Daniel Kidega
Getting page info data for:  Yabezi Kiiza
Getting page info data for:  Pumla Kisosonkole
Getting page info data for:  Annie Logiel
Getting page info data for:  James Mamawi
Getting page info data for:  Jack Maumbe Mukhwana
Getting page info data for:  Willy Mayambala
Getting page info data for:  Kibirige Mayanja
Getting page info data for:  Tamale Mirundi
Getting page info data for

Getting page info data for:  Antonio Mercader
Getting page info data for:  Pablo Mieres
Getting page info data for:  José Benito Monterroso
Getting page info data for:  Tristán Narvaja
Getting page info data for:  Gonzalo Nin Novoa
Getting page info data for:  Edgardo Novick
Getting page info data for:  María Inés Obaldía
Getting page info data for:  Yamandú Orsi
Getting page info data for:  José Brito del Pino
Getting page info data for:  Enrique Pintado
Getting page info data for:  Carlos Pita (politician)
Getting page info data for:  Carlos Quijano
Getting page info data for:  Laura Raffo
Getting page info data for:  José Rondeau
Getting page info data for:  Elbio Rosselli
Getting page info data for:  Florencio Sánchez
Getting page info data for:  José Pedro Varela
Getting page info data for:  Pedro Vaz (diplomat)
Getting page info data for:  Mateo Vidal
Getting page info data for:  Jorge Zabalza
Getting page info data for:  Ikhtiyor Abdullayev
Getting page info data for:  Adham Ahm

Getting page info data for:  Muhammad bin Yahya bin Muhammad
Getting page info data for:  Ali Muhammad Mujawar
Getting page info data for:  Nasr Taha Mustafa
Getting page info data for:  Abdul Qader Qahtan
Getting page info data for:  Mohamed Saleh Qara'a
Getting page info data for:  Mohamed Qubaty
Getting page info data for:  Rashad al-Alimi
Getting page info data for:  Abdulwahab al-Rawhani
Getting page info data for:  Mohammed al-Sabry
Getting page info data for:  Ali Abdullah Saleh
Getting page info data for:  Yahya Mohamed Abdullah Saleh
Getting page info data for:  Abdullatif Al-Sayed
Getting page info data for:  Mohammad Sewar
Getting page info data for:  Mahmoud al-Subaihi
Getting page info data for:  Saadaldeen Talib
Getting page info data for:  Ali Taysir
Getting page info data for:  Yahya Muhammad Hamid ed-Din
Getting page info data for:  Riad Yassin
Getting page info data for:  Aidarus al-Zoubaidi
Getting page info data for:  Sylvia Bambala Chalikosa
Getting page info data 

Now we will output the politicians and revision IDs to csv so this crawl will not need to be made again until we want to refresh the data.

In [97]:
revision_id_table = pd.DataFrame.from_dict(politician_revisions.items())
revision_id_table.columns = ['name', 'revision_id']
revision_id_table.to_csv('data/article_revision_id.csv')

Now we will save the list of politicians we were unable to get ORES scores for due to no revision ID.

In [99]:
no_log_list = pd.DataFrame(no_log)
no_log_list.columns = ['name']
no_log_list.to_csv('data/no_ORES_score_list.csv')

Now we will retrieve article quality predictions from ORES for all politicians we received a revision ID for. We will also create a list of all politicians we were unable to get a prediction for.

In [88]:
politician_list = list(politician_revisions.keys())
predicted_score = {}
no_score = []
for politician in politician_list:
    ARTICLE = politician
    ARTICLE_REV = politician_revisions[ARTICLE]
    print("Getting ORES scores for '%s' with revid: %d"%(ARTICLE,ARTICLE_REV))
    score = request_ores_score_per_article(ARTICLE_REV)
    try:
        predict = json.dumps(score['enwiki']['scores'][str(ARTICLE_REV)]['articlequality']['score']['prediction'])
        predicted_score[politician] = predict
    except KeyError:
        no_score.append(politician)
        print('No predicted score for ', politician)

Getting ORES scores for 'Shahjahan Noori' with revid: 1099689043
Getting ORES scores for 'Abdul Ghafar Lakanwal' with revid: 943562276
Getting ORES scores for 'Majah Ha Adrif' with revid: 852404094
Getting ORES scores for 'Haroon al-Afghani' with revid: 1095102390
Getting ORES scores for 'Tayyab Agha' with revid: 1104998382
Getting ORES scores for 'Ahmadullah Wasiq' with revid: 1109361754
Getting ORES scores for 'Aziza Ahmadyar' with revid: 1087211008
Getting ORES scores for 'Muqadasa Ahmadzai' with revid: 1082489593
Getting ORES scores for 'Mohammad Sarwar Ahmedzai' with revid: 1038918070
Getting ORES scores for 'Amir Muhammad Akhundzada' with revid: 1069322182
Getting ORES scores for 'Nasrullah Baryalai Arsalai' with revid: 1095526840
Getting ORES scores for 'Mohammad Asim Asim' with revid: 1013838830
Getting ORES scores for 'Atiqullah Atifmal' with revid: 1112407669
Getting ORES scores for 'Abdul Rahim Ayoubi' with revid: 1108886061
Getting ORES scores for 'Alhaj Mutalib Baig' with 

Getting ORES scores for 'Kastriot Berissa' with revid: 1094050369
Getting ORES scores for 'Bardhok Biba' with revid: 1110580984
Getting ORES scores for 'Dervish Biçaku' with revid: 1106794267
Getting ORES scores for 'Kostandin Boshnjaku' with revid: 1107876476
Getting ORES scores for 'Spartak Braho' with revid: 1098235893
Getting ORES scores for 'Safet Butka' with revid: 1067080697
Getting ORES scores for 'Uran Butka' with revid: 1087272778
Getting ORES scores for 'Refo Çapari' with revid: 1061419179
Getting ORES scores for 'Neritan Ceka' with revid: 1095178618
Getting ORES scores for 'Hysni Curri' with revid: 1114418989
Getting ORES scores for 'Ibrahim Dalliu' with revid: 1087026828
Getting ORES scores for 'Niazi Demi' with revid: 1104589367
Getting ORES scores for 'Abdurrahman Dibra' with revid: 1101365003
Getting ORES scores for 'Fuad Dibra' with revid: 1079550815
Getting ORES scores for 'Isuf Dibra' with revid: 1108975040
Getting ORES scores for 'Bujar Dida' with revid: 1112281152


Getting ORES scores for 'Branca Manuel Espirito Santo' with revid: 1082903108
Getting ORES scores for 'André Luís Brandão' with revid: 1092527809
Getting ORES scores for 'André Gaspar Mendes de Carvalho' with revid: 871672690
Getting ORES scores for 'Carolina Cerqueira' with revid: 1081277226
Getting ORES scores for 'Arlete Chimbinda' with revid: 1101656146
Getting ORES scores for 'Kafundanga Chingunji' with revid: 1111470901
Getting ORES scores for 'Jeremias Chitunda' with revid: 1076646666
Getting ORES scores for 'Abel Chivukuvuku' with revid: 987997249
Getting ORES scores for 'Adalberto Costa Júnior' with revid: 1107102259
Getting ORES scores for 'Adjany Costa' with revid: 1081302172
Getting ORES scores for 'Álvaro Arnaldo Craveiro' with revid: 1112653739
Getting ORES scores for 'Raul Danda' with revid: 1099408586
Getting ORES scores for 'Carlos António Fernandes' with revid: 1003396569
Getting ORES scores for 'Fiel Domingos Constantino' with revid: 1082902869
Getting ORES scores fo

Getting ORES scores for 'Gevorg Ghazaryan (politician)' with revid: 1082547991
Getting ORES scores for 'Andrias Ghukasyan' with revid: 1108345283
Getting ORES scores for 'Aram Grigoryan' with revid: 1112117000
Getting ORES scores for 'Armen Grigoryan (politician)' with revid: 1069944070
Getting ORES scores for 'Hayk Grigoryan' with revid: 1090623549
Getting ORES scores for 'Grigor Jaghetyan' with revid: 1062820993
Getting ORES scores for 'Tigran Karapetyan' with revid: 1112833785
Getting ORES scores for 'Khachatur-Bek of Mush' with revid: 1114420639
Getting ORES scores for 'Aram Kocharyan' with revid: 1107982644
Getting ORES scores for 'Davit Lokyan' with revid: 1089241404
Getting ORES scores for 'Hrant Maloyan' with revid: 1114902744
Getting ORES scores for 'Khachatur Malumian' with revid: 1114420669
Getting ORES scores for 'Aram Manukyan' with revid: 1048512708
Getting ORES scores for 'Gevorg Melik-Karagyozyan' with revid: 980519720
Getting ORES scores for 'Vahan Minakhoryan' with re

Getting ORES scores for 'Muzamil Abdullayev' with revid: 1107065506
Getting ORES scores for 'Leyla Abdullayeva' with revid: 1102138152
Getting ORES scores for 'Elchin Afandiyev' with revid: 1075526613
Getting ORES scores for 'Fazail Agamali' with revid: 1106843986
Getting ORES scores for 'Fuad Alasgarov' with revid: 1018398706
Getting ORES scores for 'Anvar Alikhanov' with revid: 1109309028
Getting ORES scores for 'Tarlan Aliyarbayov' with revid: 1112064160
Getting ORES scores for 'Dilara Aliyeva' with revid: 1109649588
Getting ORES scores for 'Ali Asadov' with revid: 1112134949
Getting ORES scores for 'Akram Aylisli' with revid: 1106344373
Getting ORES scores for 'Sahil Babayev' with revid: 1112800095
Getting ORES scores for 'Huseyngulu Baghirov' with revid: 1080007779
Getting ORES scores for 'Alovsat Bakhshiyev' with revid: 1090036474
Getting ORES scores for 'Rafael Kamil Dzhabrailov' with revid: 1100266382
Getting ORES scores for 'Matin Eynullayev' with revid: 1110146340
Getting ORE

...

Getting ORES scores for 'Mustapha Masmoudi' with revid: 1090592661
Getting ORES scores for 'Tarak Mekki' with revid: 1061580986
Getting ORES scores for 'Ahmed Mestiri' with revid: 1107510416
Getting ORES scores for 'Abir Moussi' with revid: 1107177286
Getting ORES scores for 'Naceur Ben Jaâfar' with revid: 994431648
Getting ORES scores for 'Nizar Ben Néji' with revid: 1106907647
Getting ORES scores for 'Mahmud Qabadu' with revid: 1085035197
Getting ORES scores for 'Giuseppe Raffo' with revid: 1100063840
Getting ORES scores for 'Mustapha Saheb Ettabaa' with revid: 1097001600
Getting ORES scores for 'Salah Farhat' with revid: 1080735914
Getting ORES scores for 'Saïda Sassi' with revid: 1049417844
Getting ORES scores for 'Béchir Sfar' with revid: 1065301288
Getting ORES scores for 'Tahar Sfar' with revid: 1098187195
Getting ORES scores for 'Ahmed Tlili' with revid: 1100681508
Getting ORES scores for 'Imed Trabelsi' with revid: 1109307513
Getting ORES scores for 'Tahar Zaouche' with revid:

Getting ORES scores for 'James Obita' with revid: 994193575
Getting ORES scores for 'Amos Okot' with revid: 1094002050
Getting ORES scores for 'Bosmic Otim' with revid: 1095364177
Getting ORES scores for 'Ricky Richard Anywar' with revid: 1080532652
Getting ORES scores for 'Ruth Molly Lematia' with revid: 1100462639
Getting ORES scores for 'Eric Sakwa' with revid: 1092463612
Getting ORES scores for 'Sitenda Sebalu' with revid: 1015498267
Getting ORES scores for 'Wilson Tumwine' with revid: 1036632061
Getting ORES scores for 'Sarah Nambozo Wekomba' with revid: 1027458500
Getting ORES scores for 'Oleksiy Arestovych' with revid: 1112822378
Getting ORES scores for 'Oleh Barna' with revid: 1107628544
Getting ORES scores for 'Alexander Barybin' with revid: 1076927507
Getting ORES scores for 'Andriy Beshta' with revid: 1112281458
Getting ORES scores for 'Yurii Biriukov (volunteer)' with revid: 1113311776
Getting ORES scores for 'Ivan Bisyuk' with revid: 1110372331
Getting ORES scores for 'Iho

Getting ORES scores for 'José Gervasio Artigas' with revid: 1100186329
Getting ORES scores for 'Artigas Barrios' with revid: 1101999869
Getting ORES scores for 'Sebastián Bauzá' with revid: 1108122058
Getting ORES scores for 'Mónica Bottero' with revid: 1092915419
Getting ORES scores for 'Jorge Bruni' with revid: 1101182986
Getting ORES scores for 'Guillermo Chifflet' with revid: 1101759783
Getting ORES scores for 'Guillermo Domenech' with revid: 1097158325
Getting ORES scores for 'Lorenzo Antonio Fernández' with revid: 1077548180
Getting ORES scores for 'Pedro Figari' with revid: 1096488492
Getting ORES scores for 'Liliam Kechichián' with revid: 1093592844
Getting ORES scores for 'Dámaso Antonio Larrañaga' with revid: 1095942696
Getting ORES scores for 'Juan Francisco Larrobla' with revid: 1077548411
Getting ORES scores for 'Eduardo Lorier' with revid: 1018727512
Getting ORES scores for 'Hector Luisi' with revid: 1043438229
Getting ORES scores for 'Guido Manini Ríos' with revid: 10798

Getting ORES scores for 'Chu Văn An' with revid: 1105680159
Getting ORES scores for 'Đinh Xuân Quảng' with revid: 1101628325
Getting ORES scores for 'Kiem Do' with revid: 1105970447
Getting ORES scores for 'Đỗ Quang Giai' with revid: 1106913686
Getting ORES scores for 'Hồ Đắc Điềm' with revid: 1099396979
Getting ORES scores for 'Hồ Văn Nhựt' with revid: 1106111640
Getting ORES scores for 'Hoàng Cao Khải' with revid: 1096532438
Getting ORES scores for 'Hoang Van Chi' with revid: 1101628375
Getting ORES scores for 'Huỳnh Văn Cao' with revid: 1104041830
Getting ORES scores for 'Lê Minh Hương' with revid: 1106651628
Getting ORES scores for 'Lê Quốc Minh' with revid: 1106248998
Getting ORES scores for 'Nguyễn Đức Chung' with revid: 1106670725
Getting ORES scores for 'Nguyễn Duy Quý' with revid: 1088453067
Getting ORES scores for 'Nguyễn Hợp Đoàn' with revid: 1037311684
Getting ORES scores for 'Nguyễn Quốc Cường' with revid: 1106913394
Getting ORES scores for 'Nguyễn Thị Hồng (economist)' wi

Getting ORES scores for 'Lovemore Madhuku' with revid: 1106648610
Getting ORES scores for 'Sarah Mahoka' with revid: 1109140716
Getting ORES scores for 'Alfred Makwarimba' with revid: 1092395186
Getting ORES scores for 'Washington Malianga' with revid: 1064532257
Getting ORES scores for 'Elliot Manyika' with revid: 1098102512
Getting ORES scores for 'Linda Masarira' with revid: 1106829214
Getting ORES scores for 'Jevan Maseko' with revid: 1084850620
Getting ORES scores for 'Anxious Jongwe Masuka' with revid: 1084841305
Getting ORES scores for 'Lovemore Matombo' with revid: 1105192324
Getting ORES scores for 'Bright Matonga' with revid: 1036801272
Getting ORES scores for 'Isaac Matongo' with revid: 1007345232
Getting ORES scores for 'Obedingwa Mguni' with revid: 1108676758
Getting ORES scores for 'Promise Mkwananzi' with revid: 1110541985
Getting ORES scores for 'Max Mnkandla' with revid: 1007344798
Getting ORES scores for 'Bekithemba Mpofu' with revid: 1085044064
Getting ORES scores fo

The no_score list is empty so all remaining politicians were able to receive a quality score. Now we will export the politicians and article quality to csv so this crawl will not need to be made again until we want to refresh the data.

In [96]:
pred_score_table = pd.DataFrame.from_dict(predicted_score.items())
pred_score_table.columns = ['name', 'article_quality']
pred_score_table.to_csv('data/predicted_article_quality.csv')

## Now we will combine all of the datasets!

First we will alter the country data to place region in its own row. 

In [100]:
world_pop = pd.read_csv('population_by_country_2022.csv')
world_pop.head()

,Geography,Population (millions)
0,WORLD,7963.0
1,AFRICA,1419.0
2,NORTHERN AFRICA,251.0
3,Algeria,44.9
4,Egypt,103.5


Now let's create a region column where data is only filled for countries, but region data is maintained.

In [111]:
region = []
for geo in range(len(world_pop['Geography'])):
    if world_pop['Geography'][geo].upper() == world_pop['Geography'][geo]:
        new_region = world_pop['Geography'][geo]
        region.append('')
    else:
        region.append(new_region)
world_pop['Region'] = region
world_pop.head(10)

,Geography,Population (millions),Region
0,WORLD,7963.0,
1,AFRICA,1419.0,
2,NORTHERN AFRICA,251.0,
3,Algeria,44.9,NORTHERN AFRICA
4,Egypt,103.5,NORTHERN AFRICA
5,Libya,6.8,NORTHERN AFRICA
6,Morocco,36.7,NORTHERN AFRICA
7,Sudan,46.9,NORTHERN AFRICA
8,Tunisia,11.8,NORTHERN AFRICA
9,Western Sahara,0.6,NORTHERN AFRICA


Now we will find the countries that are not in the politicians data and export a list of them into the data folder.

In [122]:
# Get the countries that are not in the politician files
no_politicians = []
for country in world_pop['Geography']:
    if country.upper() == country:
        continue
    else:
        if country not in np.unique(wiki_politicians['country']):
            no_politicians.append(country)
# save the list to txt
no_politicians = np.asarray(no_politicians)
np.savetxt('data/wp_countries-no_match.txt', no_politicians, delimiter="/n", fmt="%s")

Now we will join all the data together into one file. And save it as a csv.

In [142]:
# Merging the data together
wp_politicians_by_country = pd.merge(wiki_politicians, pred_score_table, how = 'left', on = 'name')
wp_politicians_by_country = wp_politicians_by_country.dropna(subset = ['article_quality'])
wp_politicians_by_country = pd.merge(wp_politicians_by_country, revision_id_table, how = 'left', on = 'name')
wp_politicians_by_country['revision_id'] = wp_politicians_by_country['revision_id'].astype(int)
wp_politicians_by_country = pd.merge(wp_politicians_by_country, world_pop, how = 'left', left_on = 'country',
                                    right_on = 'Geography')
# Clean the columns
wp_politicians_by_country = wp_politicians_by_country.drop(['Geography', 'url'], axis=1)
wp_politicians_by_country = wp_politicians_by_country.rename(
    columns = {'Region':'region', 'Population (millions)':'population', 'name':'article_title'})
# Export the data
wp_politicians_by_country.to_csv('data/wp_politicians_by_country.csv')